# Initialization and Optimization

## 1. Introduction

In [222]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from keras import optimizers

In [223]:
# load dataset
data = pd.read_csv("loan_final.csv", header=0)
data = data.dropna()

In [224]:
np.shape(data)

(41394, 16)

In [225]:
data.head()

,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,total_acc,total_pymnt,application_type
0,5000.0,4975.0,36 months,10.65%,162.87,B,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card,AZ,9.0,5863.155187,Individual
1,2500.0,2500.0,60 months,15.27%,59.83,C,< 1 year,RENT,30000.0,Source Verified,Charged Off,car,GA,4.0,1014.530000,Individual
2,2400.0,2400.0,36 months,15.96%,84.33,C,10+ years,RENT,12252.0,Not Verified,Fully Paid,small_business,IL,10.0,3005.666844,Individual
3,10000.0,10000.0,36 months,13.49%,339.31,C,10+ years,RENT,49200.0,Source Verified,Fully Paid,other,CA,37.0,12231.890000,Individual
4,3000.0,3000.0,60 months,12.69%,67.79,B,1 year,RENT,80000.0,Source Verified,Fully Paid,other,OR,38.0,4066.908161,Individual


In [226]:
X0 = data["loan_amnt"]
X1 = data["funded_amnt_inv"]
X2 = data["installment"]
X3 = data["annual_inc"]

# standardized inputs
X0= (X0-np.mean(X0))/(np.std(X0))
X1= (X1-np.mean(X1))/(np.std(X1))
X2= (X2-np.mean(X2))/(np.std(X2))
X3= (X3-np.mean(X3))/(np.std(X3))

In [227]:
#SAME RESULT

#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
#X0 = sc.fit_transform(X0)

In [228]:
dummy_ver = pd.get_dummies(data['verification_status'])
dummy_home = pd.get_dummies(data['home_ownership'])
dummy_emp = pd.get_dummies(data['emp_length'])

data_fin = pd.concat([X0, X1, X2, X3, dummy_ver, dummy_home, dummy_emp], axis=1)
final_data = data_fin.dropna()
y = data["total_pymnt"]
y_fin = (y-np.mean(y))/np.std(y)


In [229]:
np.shape(final_data)

(41394, 23)

In [239]:
np.random.seed(123)
model = Sequential()
model.add(Dense(7, input_dim=23, kernel_initializer='normal', activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation = 'linear'))
model.compile(optimizer="sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(final_data, y_fin, batch_size=64, epochs=30, verbose=0)

In [240]:
predictions = model.predict(final_data).reshape(-1)
MSE_final =  np.mean((predictions-y_fin)**2)
MSE_final

0.19048736797950605

In [230]:
sgd = optimizers.SGD(lr=0.01, clipnorm=1.)
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(7, input_dim=23, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error',metrics=['mse'], optimizer = sgd)
    return model

In [231]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, epochs=30, batch_size=20, verbose=1)

In [221]:
kfold = KFold(n_splits = 3, random_state = 123)
results = cross_val_score(estimator, final_data, y_fin, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/30
27596/27596 [==============================] - 3s 98us/step - loss: nan - mean_squared_error: nan
Epoch 2/30
27596/27596 [==============================] - 3s 97us/step - loss: nan - mean_squared_error: nan
Epoch 3/30
27596/27596 [==============================] - 3s 95us/step - loss: nan - mean_squared_error: nan
Epoch 4/30
27596/27596 [==============================] - 2s 88us/step - loss: nan - mean_squared_error: nan
Epoch 5/30
27596/27596 [==============================] - 2s 80us/step - loss: nan - mean_squared_error: nan
Epoch 6/30
27596/27596 [==============================] - 3s 99us/step - loss: nan - mean_squared_error: nan
Epoch 7/30
21460/27596 [======================>.......] - ETA: 0s - loss: nan - mean_squared_error: nan

KeyboardInterrupt: 

In [61]:
results

array([-1.18908199e+08, -7.83050103e+07, -5.85350814e+07])

## 8. Conclusion

https://github.com/susanli2016/Machine-Learning-with-Python/blob/master/Consumer_complaints.ipynb

https://catalog.data.gov/dataset/consumer-complaint-database

## Further reading

https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/

https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/

https://stackoverflow.com/questions/37232782/nan-loss-when-training-regression-network
https://www.springboard.com/blog/free-public-data-sets-data-science-project/

data from The Lending Club